In [363]:
import tensorflow as tf


batch_size      = 50 
classes         = tf.constant(10, tf.int8, name = "classes")
no_of_labels = 10


filter_channels = tf.Variable([64, 128, 256],name ='filter_channels')
no_of_resd_blocks = tf.Variable([3, 6, 4], name = 'no_of_resd_blocks')



flt_shape = tf.Variable(3)
dim_change_flt_shape = tf.Variable(1)
dim_change_stride = tf.Variable(2)

x = tf.placeholder(tf.float32, shape = [None, 28 * 28], name = "x")
x_rs = tf.reshape(x, shape = [-1, 28, 28, 1])

y = tf.placeholder(tf.float32, shape = [None, 10], name = "y")
y_rs = tf.reshape(y, [-1, 10])

In [364]:

def get_scale_and_gamma_variables(shape):
    scale = tf.Variable(tf.ones(shape))
    offset = tf.Variable(tf.zeros(shape))
    
    return scale, offset


def batch_normalize(data, shape):
    
    scale, offset = get_scale_and_gamma_variables(shape)
    mean, var     = tf.nn.moments(data, axes = [0, 1, 2])
    
    data_norm     = tf.nn.batch_normalization(data, mean, var, scale, offset, 0.05, name = "BN")
    
    return data_norm


In [370]:


def get_flt_channels_and_resd_blocks():
    

    return [16, 32, 64], [3, 6, 4]


#---------------------------#---------------------------------------#

def get_conv_and_pool_params_at_stage_1():
    conv_flt_shape = ([3, 3, 1, 16])
    conv_stride    = ([1, 2, 2, 1])
    pool_shape     = ([1, 2, 2, 1])
    pool_stride    = ([1, 2, 2, 1])
    

    
    return conv_flt_shape, conv_stride, pool_shape, pool_stride




#---------------------------#---------------------------------------#

def get_weight(w_name, shape, dtype):
    
    return tf.Variable(tf.truncated_normal(shape, stddev = 0.2, seed = 23, dtype = tf.float32), name = w_name)
  
    
    

#---------------------------#---------------------------------------#

def get_total_no_of_neurons(shape_of_conv_out):
    neurons = 1
    for num in shape_of_conv_out:
        if num is not None:
            neurons = neurons * num
    return neurons
    

#---------------------------#---------------------------------------#

def conv_bn_relu_stage_1(data, conv_flt_shape, stride, pool_size, pool_stride):
    
    conv_flt  = get_weight("weight1", conv_flt_shape, tf.float32)
    
    data      = tf.nn.conv2d(data, conv_flt, strides = stride, padding = "SAME")
    data_norm = batch_normalize(data, conv_flt_shape[-1])
    data_relu = tf.nn.relu(data_norm)
    data_pool = tf.nn.max_pool(data, ksize = pool_size, strides = pool_stride, padding = 'SAME')
    
    return data_pool






#---------------------------#---------------------------------------#
'''
This function is equla to one residual block in each residual layer
'''
def conv_bn_relu_resd_stage(data, conv_flt_shape, stride_1, stride_2, stride_3, padding_1, padding_2, padding_3,
                                            dim_change, in_channels, num_channels, block_no):
    
    
    with tf.variable_scope("conv_1_resd"):
        
        tf.cast(conv_flt_shape, tf.int32)
        conv_wt_1 = get_weight(("conv_wt_1_" + str(block_no) + "_"+ str(num_channels)), conv_flt_shape, tf.float32)
        conv_1    = tf.nn.conv2d(data, conv_wt_1, strides = [1, stride_1, stride_1, 1], padding = padding_1)
        bn_1      = batch_normalize(conv_1, conv_1.get_shape().as_list()[-1]) 
        relu_1    = tf.nn.relu(bn_1)
    
    with tf.variable_scope('conv_2_resd'):

        conv_flt_shape_2 = [3, 3, relu_1.get_shape().as_list()[-1], num_channels]
        conv_wt_2 = get_weight(("conv_wt_2_" + str(block_no) + "_"+ str(num_channels)), conv_flt_shape_2, tf.float32)
        conv_2    = tf.nn.conv2d(relu_1, conv_wt_2, strides = [1, stride_2, stride_2, 1], padding = padding_2)
        bn_2      = batch_normalize(conv_2, conv_2.get_shape().as_list()[-1])
        
    '''for shortcut connection between different blocks'''
    if dim_change:
        #shp       = dim_change_flt_shape
        #stride    = dim_change_stride
        shp = 1
        
        conv_wt   = get_weight(("shrt_conv_wt_" + str(block_no) + "_"+ str(num_channels)), [shp, shp, in_channels, num_channels], tf.float32)
        shrtct_in = tf.nn.conv2d(data, conv_wt, strides = [1, stride_3, stride_3, 1], padding = padding_3) 

        return tf.nn.relu(bn_2 + shrtct_in)
    
    else:
        return tf.nn.relu(bn_2 + data)
        
        
        
        
#---------------------------#---------------------------------------#    

def residual_block(r_data, num_channels, use_1_by_1_conv = False, first_block = False, block_no = 0):
    
    '''if first_block, then we have to increase the volumne
       if use_1_by_1_conv then we have to change the dimensions of the shortcut input to the addition after last block'''
    
    stride_1 = 1
    stride_2 = 1
    stride_3 = 1
    padding_1 = 'SAME'
    padding_2 = 'SAME'
    padding_3 = 'SAME'
    
    dim_change = False
    in_channels = r_data.get_shape().as_list()[-1]
    conv_flt_shape = [3, 3, in_channels, num_channels]
    
    if use_1_by_1_conv:
        stride_3 = 2
        padding_3 = 'VALID'
        dim_change = True
        conv_flt_shape = [3, 3, num_channels, num_channels]
    if first_block:
        stride_1 = 2  
        dim_change = True
        conv_flt_shape = [3, 3,  in_channels, num_channels]
        
    
    
    
    output = conv_bn_relu_resd_stage(r_data, conv_flt_shape, stride_1, stride_2, stride_3, padding_1, padding_2, 
                                          padding_3, dim_change, in_channels, num_channels, block_no)
    
    return output



#---------------------------#---------------------------------------# 

def fully_connected_layer(data, num_of_labels):
    
    data_norm = batch_normalize(data, data.get_shape().as_list()[-1])
    
    
    data_shape = data_norm.get_shape().as_list()
    total_no_of_neurons = get_total_no_of_neurons(data_shape)
    
    
    fc_wt_1 = get_weight('fc_wt_1', tf.cast([total_no_of_neurons, 10], tf.int32), tf.float32)
    fc_b_1  = get_weight('fc_b_1', [10], tf.float32)
    data_flatten = tf.reshape(data_norm, shape = [-1, total_no_of_neurons])
    
    fc_layer_out    = tf.add(tf.matmul(data_flatten, fc_wt_1), fc_b_1)
    
    return fc_layer_out
 
    
    
#---------------------------#---------------------------------------#

def perfom_average_pooling(data):
    return tf.nn.avg_pool(data, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            
    
    
#---------------------------#---------------------------------------#

def form_residual_blocks_and_fc_ouput():
    
    result_at_every_stage = []    
    use_1_by_1_conv = False
    first_block     = False
    c_flt_shape, c_stride, pool_shape, pool_stride = get_conv_and_pool_params_at_stage_1()
    x_1 = conv_bn_relu_stage_1(x_rs, c_flt_shape, c_stride, pool_shape, pool_stride)
    result_at_every_stage.append(x_1)
    
    '''
    filter_channels(64, 128, 256, 512) correspond to no_of_filters in various residual blocks
    no_of_blocks correspond to number of cn-bn-relu-cn-bn-relu blocks in each residual block
    '''    
    
    flt_channels, resd_blocks = get_flt_channels_and_resd_blocks()
    
    for flt_row in range(0, np.shape(flt_channels)[0]):
        for resd_row in range(0, np.shape(resd_blocks)[0]):
            if flt_row == resd_row:
                num_channels = flt_channels[flt_row]
                num     = resd_blocks[resd_row]
                
                for block_no in range(0, num):
                    
                    with tf.variable_scope("block_" + str(block_no), reuse = True): 
                        if block_no == 0:
                            if num_channels != 16:
                                use_1_by_1_conv = True
                                first_block     = True
                            
                            result_at_every_stage.append(residual_block(result_at_every_stage[-1], num_channels,
                                                                        use_1_by_1_conv, first_block, block_no))
                            use_1_by_1_conv = False
                            first_block = False

                        else:
                            result_at_every_stage.append(residual_block(result_at_every_stage[-1], num_channels, False, False, 
                                                                                   block_no))

                
    avg_pool_res = perfom_average_pooling(result_at_every_stage[-1]) 
    
    fc_output_layer = fully_connected_layer(avg_pool_res, classes)
    
    return fc_output_layer
                
                
                
#---------------------------#---------------------------------------#  
    
    
def ResNet_model():
    
    resnet_model        = form_residual_blocks_and_fc_ouput()
    
    cost                = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_rs, logits = resnet_model))
    train_optim         = tf.train.AdamOptimizer(learning_rate =0.001).minimize(cost)
    correctly_predicted = tf.equal(tf.argmax(resnet_model, 1), tf.argmax(y_rs, 1))
    accuracy            = tf.reduce_mean(tf.cast(correctly_predicted, tf.float32))
    
    return resnet_model, train_optim, accuracy

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler 

##########################################################
def read_data(directory_path, is_train_data):
    if is_train_data:
        mnsit_data = pd.read_csv(directory_path, header=None, low_memory = False)
        mnsit_data.drop(0, axis = 0, inplace = True)
        train_Y    = mnsit_data[0].astype(np.float32)
        mnsit_data.drop(0, axis = 1, inplace = True)
        mnsit_data = mnsit_data.astype(np.float32)

        return mnsit_data, train_Y
    
    else:
        mnsit_test = pd.read_csv(directory_path, header = None, low_memory = False)
        mnsit_test.drop(0, axis = 0, inplace = True)
        mnsit_test = mnsit_test.astype(np.float32)
        
        return mnsit_test
    
    
    
##########################################################
def split_the_train_data(train_X, train_Y):
    train_x, train_y, test_x, test_y = train_test_split(train_X, train_Y, test_size = 0.2, 
                                                        random_state = 42)
    
    return [[train_x, test_x], [train_y, test_y]]
    
##########################################################
def convert_labels_to_one_hot_encoding_format(train_y, test_y):
    one_hot_encoder = OneHotEncoder(categories = 'auto')
    train_y_encoded = one_hot_encoder.fit_transform(train_y).toarray()
    test_y_encoded  = one_hot_encoder.fit_transform(test_y).toarray()
    return train_y_encoded.astype(np.float32), test_y_encoded.astype(np.float32)



##########################################################
# def convert_input_to_image_format(row):
#     return row.values.reshape(28, 28)
def convert_input_to_image_format(data):
    output = []
    no_of_rows = np.shape(data)[0]
    for row_num in range(0, no_of_rows):
        output.append(data.iloc[row_num].values.reshape(28, 28, 1))
    
    return np.array(output).astype(np.float32)
    


##########################################################
def standardize_the_data(train_data, test_data):
    scaler = StandardScaler().fit(train_data)
    train_data_scaled = scaler.transform(train_data)
    test_data_scaled  = scaler.transform(test_data)
    return pd.DataFrame(train_data_scaled, dtype = np.float32), pd.DataFrame(test_data_scaled, dtype = np.float32)





In [ ]:
train_X, train_Y = read_data('/Users/vijay/Downloads/digit-recognizer/train.csv', True)
test_X           = read_data('/Users/vijay/Downloads/digit-recognizer/test.csv', False)

train_test_data  = split_the_train_data(train_X, train_Y)
train_x, train_y = train_test_data[0]
test_x, test_y   = train_test_data[1]

train_x_standardized, test_x_standardized = standardize_the_data(train_x, test_x)
# train_x_reshaped = train_x_standardized.apply(convert_input_to_image_format, axis = 1)

train_x_reshaped = convert_input_to_image_format(train_x_standardized)
test_x_reshaped   = convert_input_to_image_format(test_x_standardized)

train_y_encoded, test_y_encoded = convert_labels_to_one_hot_encoding_format(train_y.values.reshape(-1, 1), 
                                                                           test_y.values.reshape(-1, 1))


del(train_x)
del(test_x)
del(train_y)
del(test_y)
del(train_x_standardized)
del(test_x_standardized)

In [366]:
def run_ResNet_model():
    with tf.Session() as session:
        count = 0
        model, optimizer, accuracy = ResNet_model()
        
        session.run(tf.global_variables_initializer())
        saver = tf.train.Saver(tf.trainable_variables())
#         flt_channels = [64, 128, 256]
#         resd_blocks = [3, 6, 4]
        for epoch in range(0, 5):
            count =0
            for i in range(0, len(train_x_reshaped), batch_size):
                if count <= 100:
                    print('--------------- ' + str(count))
                    train_x = train_x_reshaped[i : i + batch_size, :]
                    train_y = train_y_encoded[i : i + batch_size, :]
                    _, accuracy_val = session.run([optimizer, accuracy], feed_dict = {x_rs : train_x, 
                                                                                      y_rs : train_y })
                else:
                    break
            
                count = count + 1
                

            if epoch % 1 == 0:
                print(epoch, accuracy_val)

    
        save_path = saver.save(session, "./resnet_artist_model")
        
        
        return model

In [ ]:
resnet_fc = run_ResNet_model()

In [ ]:
with tf.Session() as session:
    meta_graph = tf.train.import_meta_graph('/resnet_artist_model.meta')
    meta_graph.restore(session, tf.train.latest_checkpoint('./'))
    
    prediction = session.run(resnet_fc, feed_dict = {x_rs : test_x_reshaped})
    print((session.run(tf.nn.softmax(prediction))))